<a href="https://colab.research.google.com/github/ikoojos/Algorithm-Debt-Research/blob/master/LR_Count.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
from google.colab import drive
from itertools import product
from sklearn.linear_model import LogisticRegression
import importlib

# Mount Google Drive
drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive/AD Final Experiments')

# Import custom modules
from preprocessing import preprocess_data
from splitting import split_data
from utils import *
from evaluate_model import evaluate_best_model
from lr_tuning import hyperparameter_tuning


for module in ['preprocessing', 'splitting', 'utils', 'evaluate_model', 'lr_tuning']:
    importlib.reload(sys.modules[module])


Mounted at /content/drive


In [ ]:
file_path = '/content/drive/My Drive/AD Identification using SATD/liu_datset_processed.csv'
data = preprocess_data(file_path)
X_train_final, X_val, X_test, y_train_final, y_val, y_test = split_data(data)

In [ ]:
from sklearn.preprocessing import StandardScaler

param_grid = {
    'C': [0.01, 1, 10],
    'penalty': ['l2'],
    'max_iter': [1, 10, 100, 200]
}

vectorizer = CountVectorizer()
best_model, best_params, best_score = hyperparameter_tuning(X_train_final, y_train_final, X_val, y_val, param_grid, vectorizer)
evaluate_best_model(best_model, best_params, best_score, X_test, y_test)

Best parameters found: {'C': 0.01, 'penalty': 'l2', 'max_iter': 200}
Validation set accuracy: 0.8588424437299035

Test Confusion Matrix:
[[ 100    1    1   62    1   10    1   24]
 [   3   44    4   20    0    3    4   11]
 [   3    5   72   33    0    6    7    9]
 [  36   19   57 1755   12  153   29  145]
 [   0    2    1    6   10    4    0    0]
 [   9    5    8   83    2  267    7    6]
 [   2    3    7   18    1    6   95   11]
 [  34   13   10  113    2    6   11 4403]]

Test Classification Report:
                        precision    recall  f1-score   support

             ALGORITHM       0.53      0.50      0.52       200
         COMPATIBILITY       0.48      0.49      0.49        89
                DEFECT       0.45      0.53      0.49       135
                DESIGN       0.84      0.80      0.82      2206
         DOCUMENTATION       0.36      0.43      0.39        23
        IMPLEMENTATION       0.59      0.69      0.63       387
                  TEST       0.62      0